In [2]:
import torch
import torch.nn as nn
import numpy as np
import math

'''
1. Channel_weighted_Conv
   어떤 '한개의' filter 에 대해, 
   입력 [r^2, N, N] 인 feature map 의 각 channel 의 conv. 결과를 순차적으로 mapping 한 [1, rN, rN] 크기의 feature map 생성하고,
   생성한 feature map 에 rxr inner filtering 을 적용한 [1, N, N] 크기의 결과를 출력하는 함수 
'''
class Channel_weighted_Conv(nn.Module):
  def __init__(self, input_size, kernel_size, stride=1, bias=True):
    super(Channel_weighted_Conv, self).__init__()
    self.input_size = input_size 
    # input_size = r^2

    self.conv = torch.nn.Conv2d(1, 1, kernel_size, stride, padding=int((kernel_size-1)/2), bias=bias)
    # 입력 feature map [batch, r^2, N, N]에서 [batch, i, N, N] 을 입력으로 받아 1 개의 feature map 출력
    # 출력이 [batch, i, N, N] 가 되게 하기 위해 padding = int((kernel_size-1)/2)
    self.PS =  nn.PixelShuffle (upscale_factor = int(math.sqrt(input_size)))
    # upscale_factor = r
    self.inner_conv = torch.nn.Conv2d(1, 1, kernel_size=int(math.sqrt(input_size)), stride=int(math.sqrt(input_size)), padding=0, bias=bias)

    '''
    apply 함수로 initialization 하기 힘들기 때문에 class 내에서 initialization 을 수행하도록 함
    '''
    for m in self.modules():
        # 이 class 에서 정의한 self.(변수)[인스턴스 변수] 목록 (conv, PS, inner_conv) 불러오기 
            classname = m.__class__.__name__
            # 불러온 module의 class 이름
            if classname.find('Conv2d') != -1:
                torch.nn.init.kaiming_normal_(m.weight)
                if m.bias is not None:
                    m.bias.data.zero_()
            # Conv2d Layer 라면 He 방법으로 해당 Layer 의 weight 들을 초기화, bias 가 True 면 bias 를 0 으로 초기화
            elif classname.find('ConvTranspose2d') != -1:
                torch.nn.init.kaiming_normal_(m.weight)
                if m.bias is not None:
                    m.bias.data.zero_()
           # ConvTranspose2d Layer 라면 He 방법으로 해당 Layer 의 weight 들을 초기화, bias 가 True 면 bias 를 0 으로 초기화

  def forward(self, x):
    s = self.conv(x[:, 0:1, :, :])

    for i in range(1, self.input_size):
      m = self.conv(x[:, i:i+1, :, :])
      s = torch.cat([s, m], dim=1)
      
    # s.shape = [batch, r^2, N, N]
    # s = 입력 feature map의 각 channel에 대해 한개의 filter 와의 단순 convolution 결과 feature map
    
    ps = self.PS(s)
    # ps.shape = [batch, 1, rN, rN]
    # ps = 한 filter에 대해 입력 feature map 의 각 channel 가중치를 학습하기 전 feature map

    out = self.inner_conv(ps)
    # out.shape = [batch, 1, N, N]
    # out = 가중치를 학습하고 난 후 필터링 결과
    return out

'''
2. WCNN_block 
   기존의 Conv2d 와 같은 입력을 받아, 각 Channel 의 가중치 (inner_filter) 를 학습하는 과정을 추가하여
   입력과 동일한 크기의 output_size 개 만큼의 feature map 을 출력하는 New Conv2d layer
'''
class WCNN_block(nn.Module):
  def __init__(self, input_size, output_size, kernel_size):
    super(WCNN_block, self).__init__()
    self.filters = [0]*output_size

    '''
    여기서 filter 갯수(=output_size)만큼 Channel_weighted_Conv 을 list 형태로 담고,
    '''
    self.filters = nn.ModuleList([Channel_weighted_Conv(input_size, kernel_size, stride=1, bias=True) for _ in range(0, output_size)])
    self.filter_num = output_size
  
  def forward(self, x):
    '''
    여기서 각 Channel_weighted_Conv에 x(입력) 를 대입하고 나온 출력을 concatenation 해 준다.
    -> ModuleList 는 list 안의 각 Module 을 하나씩 접근 할 수 있게 해 줌
    '''
    out = self.filters[0](x)
    for i in range(1, self.filter_num):
      filtering = self.filters[i](x)
      out = torch.cat([out, filtering], dim=1)
    # out.shape = [batch, K, N, N]
    # out = 각 filter 들의 WCNN 결과의 concatenation
    return out

'''
3. ConvBlock
'''
class ConvBlock(nn.Module):
    def __init__(self, input_size, output_size, kernel_size=3, stride=1, padding=1, bias=True, activation='relu', norm=None):
        super(ConvBlock, self).__init__()
        self.conv = torch.nn.Conv2d(input_size, output_size, kernel_size, stride, padding, bias=bias)

        self.norm = norm
        if self.norm =='batch':
            self.bn = torch.nn.BatchNorm2d(output_size)
        elif self.norm == 'instance':
            self.bn = torch.nn.InstanceNorm2d(output_size)
        
        # self.bn : Conv Layer 출력에서 normalization 을 Instance 로 할지 Batch 로 할지 선택
        
        self.activation = activation
        if self.activation == 'relu':
            self.act = torch.nn.ReLU(True)
        elif self.activation == 'prelu':
            self.act = torch.nn.PReLU()
        elif self.activation == 'lrelu':
            self.act = torch.nn.LeakyReLU(0.2, True)
        elif self.activation == 'tanh':
            self.act = torch.nn.Tanh()
        elif self.activation == 'sigmoid':
            self.act = torch.nn.Sigmoid()
        
        # self.act : Conv Layer 출력 Activation Function 선택

        for m in self.modules():
            classname = m.__class__.__name__
            if classname.find('Conv2d') != -1:
                torch.nn.init.kaiming_normal_(m.weight)
                if m.bias is not None:
                    m.bias.data.zero_()
            # Conv2d Layer 라면 He 방법으로 해당 Layer 의 weight 들을 초기화, bias 가 True 면 bias 를 0 으로 초기화
            elif classname.find('ConvTranspose2d') != -1:
                torch.nn.init.kaiming_normal_(m.weight)
                if m.bias is not None:
                    m.bias.data.zero_()
           # ConvTranspose2d Layer 라면 He 방법으로 해당 Layer 의 weight 들을 초기화, bias 가 True 면 bias 를 0 으로 초기화
        
    def forward(self, x):
        if self.norm is not None:
            out = self.bn(self.conv(x))
        else:
            out = self.conv(x)

        if self.activation is not None:
            return self.act(out)
        else:
            return out

'''
4. WCNN
'''

class WCNN(nn.Module):
    def __init__(self):
        super(WCNN, self).__init__()
        
        '''
        1. layer2 만을 WCNN_block 으로 바꿈
        2. WCNN_block 의 input_size = r^2 이어야 하는 것을 유의함
        '''
        self.layer1 = ConvBlock(3, 64, kernel_size=9, stride=1, padding=4, activation='relu', norm=None)
        self.layer2 = nn.Sequential(
            WCNN_block(input_size=64, output_size=32, kernel_size=3),
            nn.ReLU(),
        )
        # WCNN_block 의 Kernel_size = 홀수 여야 출력이미지의 size 가 입력과 같아진다
        self.layer3 = self.layer3 = ConvBlock(32, 3, kernel_size=5, stride=1, padding=2, activation=None, norm=None)

    def forward(self, x):
        f1 = self.layer1(x)
        f2 = self.layer2(f1)
        y = self.layer3(f2)
        
        return y

print('finish')

finish


In [3]:
# 모델 확인

import os
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from google.colab import drive
from torchsummary import summary

drive.mount('/content/gdrive')

'''
1. GPU 설정 및 parameter print
'''
gpus_list = range(1)
cudnn.benchmark = True

cuda = True
if cuda and not torch.cuda.is_available():
    raise Exception("No GPU found, please run without --cuda")

model = WCNN()

'''
4. GPU 사용 여부 및 pretrained model 사용 여부 설정
'''
if cuda:
  model = model.cuda(gpus_list[0])
  model = torch.nn.DataParallel(model, device_ids=gpus_list)
    
summary(model, (3,32,32))

Mounted at /content/gdrive
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]          15,616
              ReLU-2           [-1, 64, 32, 32]               0
         ConvBlock-3           [-1, 64, 32, 32]               0
            Conv2d-4            [-1, 1, 32, 32]              10
            Conv2d-5            [-1, 1, 32, 32]              10
            Conv2d-6            [-1, 1, 32, 32]              10
            Conv2d-7            [-1, 1, 32, 32]              10
            Conv2d-8            [-1, 1, 32, 32]              10
            Conv2d-9            [-1, 1, 32, 32]              10
           Conv2d-10            [-1, 1, 32, 32]              10
           Conv2d-11            [-1, 1, 32, 32]              10
           Conv2d-12            [-1, 1, 32, 32]              10
           Conv2d-13            [-1, 1, 32, 32]              10
           C